In [ ]:
import pandas as pd
from tqdm.auto import tqdm

# Get data for paper

In [ ]:
path_to_ts = '/home/Mount_sda1/alexmakh/TargetScan/result_CWCS.tsv'
path_to_mirdb = '/home/Mount_sda1/alexmakh/isomiRNA/mirdb_custom_isores.tsv'
path_to_mirdb_new = '/home/Mount_sda1/alexmakh/isomiRNA/mirdb_custom_isores_errs.tsv'
path_to_mapper = '/home/Mount_sda1/alexmakh/TargetScan/input/UTR_sequences_human.txt'

In [ ]:
genes = 'Gene Symbol'
refseq = 'Refseq ID'

In [ ]:
ts = pd.read_csv(path_to_ts, sep='\t', header=None).rename(columns={0: refseq, 1: 'isomiRNA', 2: 'CWCS'})
ts = ts.sort_values('CWCS')
ts = ts.drop_duplicates([refseq, 'isomiRNA'], keep='first')
ts[['pre-miRNA', 'isomiRNA']] = ts['isomiRNA'].str.split('.').tolist()

In [ ]:
mirdb = pd.read_csv(path_to_mirdb, sep='\t').sort_values('Target Score', ascending=False)
mirdb = mirdb.drop_duplicates([genes, 'isomiRNA'], keep='first')

In [ ]:
mirdb_new = pd.read_csv(path_to_mirdb_new, sep='\t').sort_values('Target Score', ascending=False)
mirdb_new = mirdb_new.drop_duplicates([genes, 'isomiRNA'], keep='first')

In [ ]:
def pshape(df, name):
    print(f'{name}: {df.shape}')

In [ ]:
for i, j in zip([ts, mirdb, mirdb_new], ['ts', 'mirdb', 'mirdb_new']):
    pshape(i, j)

In [ ]:
mirdb = pd.concat([mirdb, mirdb_new], axis=0).sort_values('Target Score', ascending=False)
mirdb = mirdb.drop_duplicates([genes, 'isomiRNA'], keep='first')
pshape(mirdb, 'mirdb_concat')

In [ ]:
mapper = pd.read_csv(path_to_mapper, sep='\t')[[refseq, genes]].drop_duplicates()
pshape(mapper, 'mapper')
mapper.head()

In [ ]:
ts_ = ts.set_index(refseq).join(mapper.set_index(refseq)).reset_index(drop=True)
pshape(ts_, 'new ts')
ts_ = ts_[['isomiRNA', 'CWCS', genes, 'pre-miRNA']].drop_duplicates([genes, 'isomiRNA'], keep='first')
pshape(ts_, 'new ts no duplicates')
ts_.head()

In [ ]:
ts = ts_.copy()

In [ ]:
path_to_save = '/home/alexmakh/huge/alexmakh/data_to_paper/'

In [ ]:
ts.to_csv(path_to_save + 'ts_res_full.tsv', sep='\t', index=None)
mirdb.to_csv(path_to_save + 'mirdb_res_full.tsv', sep='\t', index=None)

mirdb[mirdb['Target Score'] >= 80].to_csv(path_to_save + 'mirdb_res_80_upper.tsv', sep='\t', index=None)

mirdb_mirs = set(mirdb['isomiRNA'])
ts_mirs = set(ts['isomiRNA'])
print('mirdb - ts mirnas:', len(mirdb_mirs - ts_mirs), 'ts - mirdb mirnas', len(ts_mirs - mirdb_mirs))

ts_mir_count = pd.DataFrame()

ts_ = ts.set_index('isomiRNA')
mirdb_ = mirdb.set_index('isomiRNA')

In [ ]:
ts_ = pd.read_csv(path_to_save + 'ts_res_full.tsv', sep='\t', index_col=0)
mirdb_ = pd.read_csv(path_to_save + 'mirdb_res_full.tsv', sep='\t', index_col=0)
ts_.shape, mirdb_.shape

In [ ]:
mirdb_mirs = set(mirdb_.index)
for mir in tqdm(mirdb_mirs):
    count = mirdb_.loc[mir].shape[0]
    t_ = ts_.loc[mir].sort_values('CWCS')

    if count >= t_.shape[0]:
        print('mirdb shape more than targetscan', mir)
    t_.iloc[:count].to_csv(path_to_save + 'ts_res_like_mirdb.tsv', sep='\t', mode='a', header=None)

#### Append to first line:
|isomiRNA|CWCS|Gene Symbol|pre-miRNA
|-|-|-|-|

In [ ]:
!sed -i '1 iisomiRNA\tCWCS\tGene Symbol\tpre-miRNA' /home/alexmakh/huge/alexmakh/data_to_paper/ts_res_like_mirdb.tsv

# Get TS and mirdb intersections

In [ ]:
gene_symbol = 'Gene Symbol'
isomir = 'isomiRNA'

In [ ]:
path_to_data = '/home/Mount_sda1/alexmakh/data_to_paper/'

In [ ]:
mirdb = pd.read_csv(path_to_data + 'mirdb_res_full.tsv', sep='\t', index_col=0)
mirdb

In [ ]:
ts = pd.read_csv(path_to_data + 'ts_res_full.tsv', sep='\t', index_col=0)
ts

In [ ]:
mirdb80 = pd.read_csv(path_to_data + 'mirdb_res_80_upper.tsv', sep='\t', index_col=0)
mirdb80

In [ ]:
ts80 = pd.read_csv(path_to_data + 'ts_res_like_mirdb.tsv', sep='\t', index_col=0)
ts80

In [ ]:
mirdb_mirs = set(mirdb80.index)
intersec = pd.DataFrame({}, columns=[isomir, 'ts_full', 'ts_short']).set_index(isomir)
len(mirdb_mirs)

In [ ]:
for mir in tqdm(mirdb_mirs):
    t_ = set(ts.loc[mir, gene_symbol])
    m_ = set(mirdb80.loc[mir, gene_symbol])
    intersec.loc[mir, 'ts_full'] = len(t_ & m_) / len(m_) * 100

In [ ]:
intersec.to_csv(path_to_data + 'comparasion.tsv', sep='\t')

In [ ]:
for mir in tqdm(mirdb_mirs):
    t_ = set(ts80.loc[mir, gene_symbol])
    m_ = set(mirdb80.loc[mir, gene_symbol])
    intersec.loc[mir, 'ts_short'] = len(t_ & m_) / len(m_) * 100

In [ ]:
intersec.to_csv(path_to_data + 'comparasion.tsv', sep='\t')